In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import patsy

import statsmodels.api as sm
from scipy.stats import ttest_ind
from matplotlib import rcParams

In [2]:
#Read 5 year dataset into a dataframe
df_crime = pd.read_csv("incidents-5y.csv")


In [3]:
#Remove all rows where the year is not 2012, can potentially allow for user input to choose year later
df_crime = df_crime[df_crime['year'] == 2012]
df_crime.shape

(143182, 26)

In [4]:
df_crime.head()

,id,date,year,month,day,week,dow,time,hour,is_night,...,comm_pop,council,council_pop,asr_zone,lampdist,lat,lon,desc,gctype,gcquality
654796,NaN,2012-01-01,2012,1,4383,0,0,00:00:00,0,1,...,0,NONE,0,NaN,NaN,33.137160,-117.073100,MAYHEM,cns/segment,65
654797,NaN,2012-01-01,2012,1,4383,0,0,00:00:00,0,1,...,0,NONE,0,NaN,NaN,32.779174,-117.035418,FRAUD,cns/segment,65
654798,NaN,2012-01-01,2012,1,4383,0,0,00:00:00,0,1,...,0,NONE,0,NaN,NaN,32.782489,-117.029644,FRAUD,cns/segment,65
654799,NaN,2012-01-01,2012,1,4383,0,0,00:00:00,0,1,...,0,NONE,0,NaN,NaN,32.761860,-117.046871,PETTY THEFT,cns/segment,65
654800,NaN,2012-01-01,2012,1,4383,0,0,00:00:00,0,1,...,6417,San007,147442,1.0,0.0,32.799034,-117.118333,FRAUD,cns/segment,65


In [5]:
#Look at the number of cities represented by the dataset and the relative frequencies of crimes in each
df_crime['city'].value_counts()

SndSAN    63047
SndSDO    24232
SndOCN     8695
SndCHU     8545
SndESC     6945
SndVIS     4474
SndELC     4354
SndCAR     3797
SndNAT     3605
SndLAM     3315
SndSNM     2644
SndENC     2340
SndSNT     2147
SndPOW     1120
SndLEM     1075
SndIMP     1014
SndCOR      893
SndSOL      485
SndDEL      455
Name: city, dtype: int64

In [6]:
#Read in the dataset which contains the full city names
df_code = pd.read_csv('city_codes.csv')
df_code.head()

,type,code,code v2,scode,name,lon,lat,area
0,city,SndSDO,CaSanSDC,CN,S.D. County,-116.658688,33.059591,9.949928e+10
1,city,SndSAN,CaSanSAN,SD,San Diego,-117.122705,32.830321,9.551285e+09
2,city,SndCHU,CaSanCHU,CV,Chula Vista,-117.014424,32.628103,1.453371e+09
3,city,SndOCN,CaSanOCN,OC,Oceanside,-117.308368,33.224657,1.175111e+09
4,city,SndPOW,CaSanPOW,PW,Poway,-117.020127,32.987160,1.090701e+09


In [7]:
def get_city_name(row):
    city_code = row['city']
    df_city_name = df_code.loc[df_code['code'] == city_code, 'name']
    city_name = df_city_name.to_string(index=False)
    return city_name

In [8]:
#Map the city code (e.g. SNDSAN) to the full city name (e.g. San Deigo) 
#for each crime incident in df_crime
#df_crime['city_name'] = df_code.loc[df_code['code'] == df_crime['city'].str, 'name']
df_crime['city_name'] = df_crime.apply(get_city_name, axis=1)

In [9]:
df_crime.head()

,id,date,year,month,day,week,dow,time,hour,is_night,...,council,council_pop,asr_zone,lampdist,lat,lon,desc,gctype,gcquality,city_name
654796,NaN,2012-01-01,2012,1,4383,0,0,00:00:00,0,1,...,NONE,0,NaN,NaN,33.137160,-117.073100,MAYHEM,cns/segment,65,Escondido
654797,NaN,2012-01-01,2012,1,4383,0,0,00:00:00,0,1,...,NONE,0,NaN,NaN,32.779174,-117.035418,FRAUD,cns/segment,65,La Mesa
654798,NaN,2012-01-01,2012,1,4383,0,0,00:00:00,0,1,...,NONE,0,NaN,NaN,32.782489,-117.029644,FRAUD,cns/segment,65,La Mesa
654799,NaN,2012-01-01,2012,1,4383,0,0,00:00:00,0,1,...,NONE,0,NaN,NaN,32.761860,-117.046871,PETTY THEFT,cns/segment,65,La Mesa
654800,NaN,2012-01-01,2012,1,4383,0,0,00:00:00,0,1,...,San007,147442,1.0,0.0,32.799034,-117.118333,FRAUD,cns/segment,65,San Diego


In [10]:
#Read the ABC licenses dataset into a new dataframe
df_alcohol = pd.read_csv('abc_licenses_sdcounty.csv')
df_alcohol = df_alcohol.drop(['city'], 1)
        

In [11]:
import copy
#function to extract the city name from the address
def get_city(row):
    address = row['premisesaddress']
    address_components = address.split(", ")
    string = address_components[-2].title()
    city_name = copy.deepcopy(string)
    return city_name
    


In [12]:
# get the city the ABC license is located from the address and dedicate a column to storing the city name
df_alcohol['city_name'] = df_alcohol.apply(get_city, axis=1)
df_alcohol.head()

,last_date,OGC_FID,GEOMETRY,licenses_id,scrape_date,licenseno,status,licensetype,issuedate,exprdate,...,y,neighborhood,community,council,tract,business,mailaddress,gcquality,gctype,city_name
0,2013-07-09T,1,NaN,NaN,2013-07-09T,413,ACTIVE,51,1951-07-20T,2014-03-31T,...,1.864266e+06,NONE,NONE,NONE,0076.00,NaN,NaN,0,cns/address,San Diego
1,2013-07-09T,2,NaN,NaN,2013-07-09T,1414,ACTIVE,41,1978-06-05T,2014-04-30T,...,1.914461e+06,NONE,NONE,NONE,0083.59,Capriccio Italian Restaurant,NaN,0,cns/address,San Diego
2,2013-07-09T,3,NaN,NaN,2013-07-09T,1419,ACTIVE,21,1964-08-18T,2013-09-30T,...,1.836128e+06,NONE,NONE,NONE,0035.01,Ocean View Liquor,NaN,0,cns/address,San Diego
3,2013-07-09T,4,NaN,NaN,2013-07-09T,2498,ACTIVE,21,1978-01-01T,2013-08-31T,...,1.853602e+06,NONE,NONE,NONE,0024.01,Market Place The,NaN,0,cns/address,San Diego
4,2013-07-09T,5,NaN,NaN,2013-07-09T,3803,ACTIVE,48,1977-05-18T,2014-03-31T,...,1.872476e+06,NONE,NONE,NONE,0079.08,Silver Fox Associates Inc,NaN,0,cns/address,San Diego


In [13]:
#Get a brief overview of the data (filtered by the full city name)
crime_counts = df_crime['city_name'].value_counts()
alcohol_counts = df_alcohol['city_name'].value_counts()
print(crime_counts)
print(alcohol_counts)

print(len(crime_counts))
print(len(alcohol_counts))

San Diego         63047
S.D. County       24232
Oceanside          8695
Chula Vista        8545
Escondido          6945
Vista              4474
El Cajon           4354
Carlsbad           3797
National City      3605
La Mesa            3315
San Marcos         2644
Encinitas          2340
Santee             2147
Poway              1120
Lemon Grove        1075
Imperial Beach     1014
Coronado            893
Solana Beach        485
Del Mar             455
Name: city_name, dtype: int64
San Diego         2661
Chula Vista        291
Oceanside          258
Escondido          256
El Cajon           222
Carlsbad           185
Encinitas          157
Vista              157
San Marcos         154
La Mesa            146
National City      102
Poway               88
Santee              79
Coronado            67
Imperial Beach      48
Solana Beach        44
Del Mar             36
Lemon Grove         36
Name: city_name, dtype: int64
19
18


In [14]:
#remove S.D. County from dataframe as that city is not represented in the ABC licenses data
df_crime = df_crime[df_crime['city_name'] != 'S.D. County']
crime_counts = df_crime['city_name'].value_counts()

In [15]:
#function to get the total number of crimes for each city
def get_total_crime(row):
    city_name = row['city_name']
    return crime_counts[city_name]

#function to get the total number of ABC licenses for each city
def get_total_alcohol(row):
    city_name = row['city_name']
    return alcohol_counts[city_name]
    

In [16]:
#aggregate data into a single dataframe for analyses

df = df_alcohol.loc[:, ['city_name']].drop_duplicates()

df.loc[:,'total_crime'] = df.apply(get_total_crime, axis=1)
df.loc[:, 'total_alcohol'] = df.apply(get_total_alcohol, axis=1)

df

,city_name,total_crime,total_alcohol
0,San Diego,63047,2661
7,National City,3605,102
8,Del Mar,455,36
10,Escondido,6945,256
12,Encinitas,2340,157
19,Imperial Beach,1014,48
20,Oceanside,8695,258
22,Lemon Grove,1075,36
23,Carlsbad,3797,185
27,Chula Vista,8545,291


In [42]:
#get the population per city
#source: U.S. Census Bureau 2010
df['city_pop'] = [1307402, 58582, 4161, 143191, 59518, 26324, 167086, 25320, 
                 105328, 243916, 99478, 93834, 57065, 53413, 47811, 18912,
                12867, 83781]

In [41]:
#break up the total number of crime counts for each city by the type of crime
df_crime_types = df_crime[['city_name', 'desc']]
df_stat = df_crime_types['desc'].value_counts()
print(len(df_stat))
df

555


,city_name,total_crime,total_alcohol,FRAUD,city_pop
0,San Diego,63047,2661,3659,1307402
7,National City,3605,102,86,58582
8,Del Mar,455,36,15,4161
10,Escondido,6945,256,401,143191
12,Encinitas,2340,157,106,59518
19,Imperial Beach,1014,48,57,26324
20,Oceanside,8695,258,379,167086
22,Lemon Grove,1075,36,55,25320
23,Carlsbad,3797,185,141,105328
27,Chula Vista,8545,291,647,243916


In [31]:
def get_crime_types(row, crime_type):
    df_counts = df_crime.loc[(df_crime['city_name'] == row['city_name']) & (df_crime['desc'] == crime_type)]
    #print(df_counts)
    return len(df_counts)
    

In [32]:
df.loc[:, 'FRAUD'] = df.apply(get_crime_types, args=('FRAUD',), axis=1)


In [33]:
df

,city_name,total_crime,total_alcohol,FRAUD
0,San Diego,63047,2661,3659
7,National City,3605,102,86
8,Del Mar,455,36,15
10,Escondido,6945,256,401
12,Encinitas,2340,157,106
19,Imperial Beach,1014,48,57
20,Oceanside,8695,258,379
22,Lemon Grove,1075,36,55
23,Carlsbad,3797,185,141
27,Chula Vista,8545,291,647
